## Documentation

To read more about the search by Query DSL, visit the docs [Paginate search results](https://www.elastic.co/guide/en/elasticsearch/reference/current/paginate-search-results.html), [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-your-data.html) and [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html).

![search_paginate_docs](../images/search_paginate_docs.png)


## Connect to ElasticSearch


In [1]:
import { Client } from "npm:@elastic/elasticsearch";
import { load } from "https://deno.land/std/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });

const client = new Client({
  node: env.ELASTICSEARCH_NODE,
  auth: {
    apiKey: env.ELASTICSEARCH_API_KEY,
  },
});

const info = await client.info();
console.log(info);


{
  name: "fead23d3120d",
  cluster_name: "docker-cluster",
  cluster_uuid: "v3fUyW9OReext6IjPiOCqg",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [2]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

Let's index the documents.


In [ ]:
import data from "../data/dummy_data_2.json" with { type: "json" };

function repeatArrayThousandTime(arr: any[]) {
  const result = [];
  for (let i = 0; i < 1000; i++) {
      result.push(...arr);
  }
  return result;
}

const dummyData = repeatArrayThousandTime(data);

console.log(dummyData.length);


5000


Since, we have duplicated the dummy data so much. Let's use the `bulk API` since we learned it before to index all those documents rapidly.


In [4]:
const operations = [];
for (const document of dummyData) {
  operations.push({
    index: {
      _index: "my_index",
    },
  });
  operations.push(document);
}

await client.bulk({
  operations,
});


{
  errors: false,
  took: 400,
  items: [
    {
      index: {
        _index: "my_index",
        _id: "vMiuAZYBo7TaDttMnVMn",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 0,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "vciuAZYBo7TaDttMnVMn",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 1,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "vsiuAZYBo7TaDttMnVMn",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 2,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "v8iuAZYBo7TaDttMnVMn",
        _version: 1,
        result: "created",
       

## Searching


### 1. Size + From

By default, you cannot use from and size to page through more than `10,000 hits`. This limit is a safeguard set by the `index.max_result_window` index setting. If you need to page through more than 10,000 hits, use the `search_after` parameter instead.

In this example, we perform a search that retrieves 10 documents, starting from the 11th document (i.e., skipping the first 10 results). This demonstrates pagination using the `size` and `from` parameters.


In [6]:
const response = await client.search({
  index: "my_index",
  query: {
    match_all: {},
  },
  size: 10,
  from: 10,
});

for (const hit of response.hits.hits) {
  console.log(hit._source);
}


{
  message: "A rare document featuring the vital keyword.",
  age: 29,
  price: 175.5
}
{
  message: "Surprising keyword found deep in the content.",
  age: 41,
  price: 138
}
{
  message: "Important keyword hidden within this snippet.",
  age: 33,
  price: 112.75
}
{
  message: "Final entry still includes the essential keyword.",
  age: 26,
  price: 199.9
}
{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}
{
  message: "A rare document featuring the vital keyword.",
  age: 29,
  price: 175.5
}
{
  message: "Surprising keyword found deep in the content.",
  age: 41,
  price: 138
}
{
  message: "Important keyword hidden within this snippet.",
  age: 33,
  price: 112.75
}
{
  message: "Final entry still includes the essential keyword.",
  age: 26,
  price: 199.9
}
{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}


### 2. Timeout

This example shows how to set a timeout for the search query. If the query takes longer than the specified `10s` (10 seconds), it will be aborted.

In [8]:
await client.search({
  index: "my_index",
  query: {
    match: {
      message: "yes, keyword included",
    },
  },
  timeout: "10s",
});


{
  took: 11,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 5000, relation: "eq" },
    max_score: 3.5346906,
    hits: [
      {
        _index: "my_index",
        _id: "wMiuAZYBo7TaDttMnVMn",
        _score: 3.5346906,
        _source: {
          message: "Randomized result: yes, keyword included.",
          age: 38,
          price: 149.25
        }
      },
      {
        _index: "my_index",
        _id: "xciuAZYBo7TaDttMnVMn",
        _score: 3.5346906,
        _source: {
          message: "Randomized result: yes, keyword included.",
          age: 38,
          price: 149.25
        }
      },
      {
        _index: "my_index",
        _id: "ysiuAZYBo7TaDttMnVMn",
        _score: 3.5346906,
        _source: {
          message: "Randomized result: yes, keyword included.",
          age: 38,
          price: 149.25
        }
      },
      {
        _index: "my_index",
        _id: "z8iuAZYBo7TaDttMnVMn",
   